In [1]:
# !pip install pytube
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import os
import pandas as pd
from moviepy.editor import *
import shutil

# download videos from URL

In [ ]:

try:
    from pytube import YouTube
    from pytube import Playlist
except Exception as e:
    print('Module is not there!!...',e)
    sys.exit()

# First get the video link from the csv file

try:
    # path = input("Enter the path: ")
    df = pd.read_csv('/content/train.csv')
    def link():

        classes = ['breakdancing','celebrating','clay pottery making',
                   'eating ice cream','golf driving','push up',
                   'raising eyebrows','riding scooter','shaking head','welding']
        all_link = []
        for i in range(len(classes)):
            new_df = df[df['label'] == classes[i]]      # get the links with matching label
            all_link.append(list(new_df['youtube_id']))

        return all_link

except:
    print('invalid path..!!')
    sys.exit()


link_list = link()

# Download the videos

main_url = 'https://www.youtube.com/watch?v='

enter_path = input("Please enter the path where you want to download the Videos:.....")

if not os.path.exists(enter_path +'/'+'videos'):   #this create video folder where you want to downlod the videos
    os.makedirs(enter_path +'/'+'videos')

main_path = enter_path +"/"+'videos'

for i in range(len(link_list)):                          #this create the folders for each class.
    if not os.path.exists(main_path + '/' + str(i)):
        os.makedirs(main_path + '/' + str(i))


want_videos = int(input("Enter how many videos you want to download for each class:....."))

for i in range(len(link_list)):
    k = 0
    for j in range(len(link_list[i])):

        downloaded_video = os.listdir(main_path+'/'+str(i)) # Find all the video name in current "i" folder.

        try:
            if k < want_videos:
                if str(link_list[i][j] + '.mp4') not in downloaded_video:  # "If" the video is not in the folder then download  otherwise go to "else".
                    url = main_url + str(link_list[i][j])
                    stream = YouTube(url).streams.first()
                    print("Downloading...!", i, str(link_list[i][j]))
                    print(url)
                    stream.download(output_path=main_path+'/'+str(i), filename=str(link_list[i][j]))

                else:
                    print("You have this video: ",i, str(link_list[i][j]))
                k = k + 1


        except:
            print('Dose not found this video: ',url)
            continue


In [4]:
N = 0 
for dirpath, dirnames, filenames in os.walk('/content/drive/MyDrive/videos/'):
    N += len(filenames)
    print("Files in", dirpath, len(filenames))

print("Total files", N)

Files in /content/drive/MyDrive/videos/ 0
Files in /content/drive/MyDrive/videos/0 22
Files in /content/drive/MyDrive/videos/1 22
Files in /content/drive/MyDrive/videos/2 22
Files in /content/drive/MyDrive/videos/3 22
Files in /content/drive/MyDrive/videos/4 22
Files in /content/drive/MyDrive/videos/5 22
Files in /content/drive/MyDrive/videos/6 22
Files in /content/drive/MyDrive/videos/7 22
Files in /content/drive/MyDrive/videos/8 16
Files in /content/drive/MyDrive/videos/9 12
Total files 204


# trim videos according to to start and end time in CSV file


In [ ]:
classified_video_folders_path = r"/content/drive/MyDrive/videos"

classified_video_folders = os.listdir(classified_video_folders_path)
folder_name = "trim_video"

# if the folder (folder_name) is already exist then first remove it.
for i in range(len(classified_video_folders)):
    if os.path.exists(classified_video_folders_path + "/" + str(i) + '/' + folder_name):
        shutil.rmtree(classified_video_folders_path + "/" + str(i) + '/' + folder_name)

# get the name of all the videos with extension .mp4
link_list_mp4 = []
for i in range(len(classified_video_folders)):
    link = os.listdir(classified_video_folders_path + '/' + str(i))
    link_list_mp4.append(link)

# remove the extention .mp4
# link_list = []
# for i in link_list_mp4:
#     temp_list = []
#     for j in i:
#         temp_list.append(j[:-3])
#     link_list.append(temp_list)
# link_list

# get the start and end time list of each video link
df = pd.read_csv('/content/train.csv')
time_start, time_end = [], []

for i in link_list_mp4:
    temp_time_start, temp_time_end = [], []
    for j in i:
        new_df=df[df['youtube_id'] == str(j)]        
        new_df = df.dropna(subset=['time_start','time_end'])  
# print(new_df)
        temp_time_start.append(new_df['time_start'].iloc[0])
        temp_time_end.append(new_df['time_end'].iloc[0])

    time_start.append(temp_time_start)
    time_end.append(temp_time_end)


# print(time_start,time_end)
# print(len(time_start),len(time_end))
# # print(link_list[9][19], time_start[9][19], time_end[9][19])


# trim the video and save in a propre directory

for i in range(len(link_list_mp4)):
    folder_name = "trim_video"
    if not os.path.exists(classified_video_folders_path + "/" + str(i)  + '/' + folder_name):
        os.makedirs(classified_video_folders_path + "/" + str(i)  + '/' + folder_name)

    new_video_path = classified_video_folders_path + "/" + str(i)  + '/' + folder_name
    for j in range(len(link_list_mp4[i])):
        path_of_each_video = classified_video_folders_path + "/" + str(i) +'/'+ str(link_list_mp4[i][j])+".mp4"
        try:
            vid = VideoFileClip(path_of_each_video)
            duration = int(vid.duration)
            # If else statement is used here because if the len of required clip is greater than actual len of video
            # so we have edit that length using duratuin parameter
            if time_end[i][j] <= duration:
                trim_video = vid.subclip(int(time_start[i][j]), int(time_end[i][j]))
                trim_video.write_videofile(new_video_path+'/'+str(j)+str(link_list_mp4[i][j])+'.mp4')
            else:
                trim_video = vid.subclip(int(time_start[i][j]), duration)
                trim_video.write_videofile(new_video_path + '/' + str(j) + str(link_list_mp4[i][j]) + '.mp4')

            os.rename(new_video_path+'/'+str(j)+str(link_list_mp4[i][j])+'.mp4', new_video_path+'/'+str(link_list_mp4[i][j])+'.mp4')  #rename each video
        except:
            print(link_list_mp4[i][j]," this video is not properly downloaded or unavailable in the ",i," folder")

